In [1]:
import os
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split, Dataset, Subset
from torchvision import transforms, models
import pytorch_lightning as pl
from pytorch_lightning.utilities.model_summary import ModelSummary
from PIL import Image
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingLR
import wandb
import torchmetrics

C:\Users\jaydenpark\Anaconda3\envs\RL\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [2]:
wandb.login()
wandb.init(
      mode='disabled',
      # Set the project where this run will be logged
      project="histopathologic-cancer-classifier", 
      # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
      name="Test4", 
      # Track hyperparameters and run metadata
      config={
      "learning_rate": 0.01,
      "data_size": 1,
      })

wandb: Currently logged in as: respwill. Use `wandb login --relogin` to force relogin


In [3]:
# backbone = models.resnet50(weights='DEFAULT')
# num_filters = backbone.fc.in_features
# layers = list(backbone.children())[:-1]
# self.feature_extractor = nn.Sequential(*layers)
# self.classifier = nn.Linear(num_filters, num_target_classes)

In [4]:
class ImageEncoder(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=5, kernel_size=3, stride=2),
            nn.ReLU(),
#             nn.BatchNorm2d(),
#             nn.Dropout(0.25),
            nn.Conv2d(in_channels=5, out_channels=7, kernel_size=5, stride=2),
            nn.ReLU(),
#             nn.BatchNorm2d(),
#             nn.Dropout(0.25),
            nn.Conv2d(in_channels=7, out_channels=10, kernel_size=3, stride=2),
            nn.ReLU(),
#             nn.BatchNorm2d(),
#             nn.Dropout(0.25)
#             nn.MaxPool2d(kernel_size=3,stride=1),
        )
        n_sizes = self._get_conv_output((3,96,96))
        # flatten and 
        self.dense = nn.Sequential(
            nn.Linear(n_sizes,3000),
#             nn.Dropout(0.25),
#             nn.Linear(15000,3000),
#             nn.Dropout(0.25),
            nn.Linear(3000,200),
#             nn.Dropout(0.25),
            nn.Linear(200,1)
#             nn.Linear(500,100),
#             nn.Linear(100,1)
        )
        
#         self.fc2 = nn.Linear(13000,6000)
#         self.fc3 = nn.Linear(6000,300)
#         self.decoder = nn.Sequential(nn.Sigmoid())
        self.accuracy = torchmetrics.Accuracy(task='binary')
    
    def forward(self, x):
        x = self.encoder(x)
        x = torch.flatten(x)
        x = self.dense(x)
        return torch.sigmoid(x)
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        pred = torch.argmax(y_hat)
        loss = F.binary_cross_entropy(y_hat, y)
        sch = self.lr_schedulers()
        sch.step()
        return {"loss":loss, 'pred':pred, 'y':y[0]}
    
    def training_epoch_end(self, outputs):
        avg_loss = torch.stack([x['loss'] for x in outputs]).mean()
        pred = torch.stack([x['pred'] for x in outputs]).to(device=torch.device('cuda'))
        y = torch.stack([x['y'] for x in outputs])
        avg_acc = self.accuracy(pred, y)
        self.log("train_loss", avg_loss, prog_bar=True)
        self.log("train_acc", avg_acc, prog_bar=True)
        wandb.log({'train_loss': avg_loss, 'learning_rate': self.lr_schedulers().get_last_lr()[0], 'train_acc':avg_acc})
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        pred = torch.argmax(logits)
        val_loss = F.binary_cross_entropy(y_hat, y)
        return {'val_loss':val_loss,'pred':pred, 'y':y[0]}
    
    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        y_hat = torch.stack([x['pred'] for x in outputs]).to(device=torch.device('cuda'))
        y = torch.stack([x['y'] for x in outputs])
        avg_acc = self.accuracy(y_hat, y)
        self.log("val_loss", avg_loss, prog_bar=True)
        self.log("val_acc", avg_acc, prog_bar=True)
        wandb.log({'val_loss': avg_loss, 'val_acc':avg_acc})
        
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=wandb.config['learning_rate'])
        scheduler = CosineAnnealingLR(optimizer, T_max=100, eta_min=0.00001)
        return [optimizer], [scheduler]
    
    def _get_conv_output(self, shape):
        batch_size = 1
        input = torch.autograd.Variable(torch.rand(batch_size, *shape))


        output_feat = self.encoder(input) 
        n_size = output_feat.data.view(batch_size, -1).size(1)
        return n_size


In [5]:
class CancerDataset(Dataset):
    def __init__(self, label_dir, img_dir):
        self.img_labels = pd.read_csv(label_dir)
        self.img_dir = img_dir
        self.transform = transforms.ToTensor()
    
    def __len__(self):
        return len(self.img_labels)
    
    def __getitem__(self, idx):
        # label file has two columns: id, label
        # to read each images from dir
        img_path = self.img_dir + '/'+ self.img_labels.iloc[idx,0] + '.tif'
        # read image as numpy array and normalize it
        image = Image.open(img_path)
        image_array = self.transform(image) / 255.0
        
        # read label
        label = self.img_labels.iloc[idx,1].astype('float32')
        
        # return image array and label
        return image_array, label

In [6]:
data = CancerDataset(label_dir=r'D://histopathologic-cancer-detection/train_labels_balance.csv',
                          img_dir=r'D://histopathologic-cancer-detection/train')

In [7]:
train_data = pd.read_csv(r'D://histopathologic-cancer-detection/train_labels.csv')

In [8]:
train_n = int(len(data) * wandb.config['data_size'] * 0.7)
val_n = int(len(data) * wandb.config['data_size'] * 0.3)

In [9]:
train_n, val_n

(124763, 53470)

In [10]:
def train_val_dataset(dataset, train_split=0.75, val_split=0.25):
    train_idx, val_idx = train_test_split(list(range(len(dataset))), train_size=train_split, test_size=val_split)
    datasets = {}
    datasets['train'] = Subset(dataset, train_idx)
    datasets['val'] = Subset(dataset, val_idx)
    return datasets

In [11]:
dataset = train_val_dataset(data, train_split=train_n, val_split=val_n)

In [12]:
train_dataloader = DataLoader(dataset['train'])
val_dataloader = DataLoader(dataset['val'])

In [13]:
im_encoder = ImageEncoder()

In [14]:
ModelSummary(im_encoder, max_depth=-1)

   | Name      | Type           | Params
----------------------------------------------
0  | encoder   | Sequential     | 1.7 K 
1  | encoder.0 | Conv2d         | 140   
2  | encoder.1 | ReLU           | 0     
3  | encoder.2 | Conv2d         | 882   
4  | encoder.3 | ReLU           | 0     
5  | encoder.4 | Conv2d         | 640   
6  | encoder.5 | ReLU           | 0     
7  | dense     | Sequential     | 3.6 M 
8  | dense.0   | Linear         | 3.0 M 
9  | dense.1   | Linear         | 600 K 
10 | dense.2   | Linear         | 201   
11 | accuracy  | BinaryAccuracy | 0     
----------------------------------------------
3.6 M     Trainable params
0         Non-trainable params
3.6 M     Total params
14.420    Total estimated model params size (MB)

In [15]:
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    dirpath='D://histopathologic-cancer-detection/trained_model', filename='{epoch}-{train_loss:.4f}-{val_loss:.4f}', 
    monitor="val_loss", mode="min", save_top_k=5
)

In [16]:
trainer = pl.Trainer(accelerator='gpu',
                    max_epochs=200,
                    callbacks=[checkpoint_callback])

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(im_encoder, train_dataloader, val_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type           | Params
--------------------------------------------
0 | encoder  | Sequential     | 1.7 K 
1 | dense    | Sequential     | 3.6 M 
2 | accuracy | BinaryAccuracy | 0     
--------------------------------------------
3.6 M     Trainable params
0         Non-trainable params
3.6 M     Total params
14.420    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

C:\Users\jaydenpark\Anaconda3\envs\RL\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
C:\Users\jaydenpark\Anaconda3\envs\RL\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]